In [1]:
!pip install gdown

# Kolekcja ponad 100 tys. nagłówków z polskich portali informacyjnych z października-listopada 2020 r.
!rm news_pl.jsonl
!gdown https://drive.google.com/uc?id=1w8sx3Rs2W1fZgSnJmsKKph0oHueH0yww

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9681 sha256=f12d1d9b6b278598e4dccbe0a8dd0beac33a6093eba430b269fe4c3b68115c77
  Stored in directory: /Users/ppez/Library/Caches/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
Successfully built gdown
You should consider upgrading via the '/Users/ppez/anaconda3/bin/python -m pip install --upgrade pip' command.
rm: news_pl.jsonl: No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1w8sx3Rs2W1fZgSnJmsKKph0oHueH0yww
To: /private/tmp/news_pl.jsonl
42.7MB [00:03, 13.2MB/s]


In [2]:
# Wczytujemy dokumenty jako listę słowników. Każdy wiersz to jeden dokument.

import json 


def read_json_documents(jsonl):
    docs = []
    with open(jsonl,'r') as file:
        for line in file:
            if(len(line.strip())>0):
                docs.append(json.loads(line))
    return docs


headlines_pl = read_json_documents('news_pl.jsonl')
print(len(headlines_pl))
print(headlines_pl[0])


116056
{'id': '81d72ff3d226249c001dd3515a413a60', 'language': 'pl', 'source': 'wirtualnemedia.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Aleksandra Budka po 8 latach rozstaje się z radiową Trójką', 'url': 'https://www.wirtualnemedia.pl/artykul/aleksandra-budka-po-8-latach-rozstaje-sie-z-radiowa-trojka', 'category': ['news']}


In [3]:
# 500 tys. artykułów z polskiej Wikipedii
!rm wiki_pl.jsonl.gz
!rm wiki_pl.jsonl

!gdown https://drive.google.com/uc?id=1go0J35NtymB-cdogrLM3YyTsyCMCMPw0
!gunzip wiki_pl.jsonl.gz

wiki_docs = read_json_documents('wiki_pl.jsonl')

print(len(wiki_docs))
print(wiki_docs[0])

rm: wiki_pl.jsonl.gz: No such file or directory
rm: wiki_pl.jsonl: No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1go0J35NtymB-cdogrLM3YyTsyCMCMPw0
To: /private/tmp/wiki_pl.jsonl.gz
25.5MB [00:01, 13.3MB/s]
500000
{'id': '931674', 'title': 'Garnizony Armii Radzieckiej na terytorium Polski', 'category': ['Północna_Grupa_Wojsk_Armii_Radzieckiej', 'Garnizony_wojska_rosyjskiego', 'III_Rzeczpospolita', 'Stosunki_polsko-sowieckie', 'Stosunki_polsko-rosyjskie'], 'language': 'pl', 'published': '2016-01-01T02:57:20Z', 'url': 'https://pl.wikipedia.org/wiki/Garnizony Armii Radzieckiej na terytorium Polski'}


In [5]:
# Przeszukiwanie sekwencyjne 'brute-force'

def bf_search(documents, strings):
    matching_docs = []
    for doc in documents:
        for s in strings:
            if s in doc['title']:
                matching_docs.append(doc)
    print("\n---------------------------\nTotal documents found: {}\n---------------------------\n"
          .format(len(matching_docs)))
            
    return matching_docs
def print_docs(documents, fields):

    for di, document in enumerate(documents):
        print("{}. {}\t{}".format(di+1, *([document[f] for f in fields])))

matching = bf_search(documents = headlines_pl,strings=['samolot'])
print_docs(matching[:5], ['id','title'])



---------------------------
Total documents found: 107
---------------------------

1. e50d4cc63e508bba38d9d125b3ded190	Wrocław: 36 hiszpańskich studentów z koronawirusem. Przylecieli jednym samolotem
2. ec4498474b4a4d0e893ece67045d9ed2	Chora na nowotwór wyruszyła na leczenie rządowym samolotem
3. 01e5965aa2a2a1da9925345eed4bd5df	18-letnia Julia poleciała do USA rządowym samolotem. "Jesteśmy z Tobą!"
4. 0843f4d8a46fc11f96c99c8b0af0b5a0	Koronawirus. 41-letnia Polka zatrzymana na lotnisku Schiphol. Gryzła i kopała załogę samolotu
5. f271eedb333a5f86f3a276afd3064e4c	Koniec latania "na wypasie". Przygotujcie się na cięcia kosztów i niewygody w samolotach


In [6]:
# Lematyzacja dokumentów w SpaCy
# !pip install -U spacy
# !python -m spacy validate
# 
!pip install spacy-nightly --pre
!python -m spacy download pl_core_news_sm


import spacy
nlp_pl = spacy.load("pl_core_news_sm")

doc = nlp_pl(headlines_pl[0]['title'])
print([w.lemma_ for w in doc])




     |████████████████████████████████| 10.8 MB 2.4 MB/s eta 0:00:01
  Using cached packaging-20.8-py2.py3-none-any.whl (39 kB)
     |████████████████████████████████| 448 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 222 kB 3.1 MB/s eta 0:00:01
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
     |████████████████████████████████| 959 kB 3.3 MB/s eta 0:00:01
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
     |████████████████████████████████| 113 kB 2.0 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=106272 sha256=6ff58864ca51c57b820970afdcc946e810bc238519b968fa704dede728011c74
  Stored in directory: /Users/ppez/Library/Caches/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: packaging
    Found existing installation: packaging 17.1
    Uninstalling packaging-17.1:
      Successfully uninstalled packaging-17.1
  Attem

/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorboard/co

  Created wheel for pl-core-news-sm: filename=pl_core_news_sm-3.0.0a0-py3-none-any.whl size=56922788 sha256=1ab2e2e2ebe1c52630af0f7c0b5ac6bcd122b5012ed67cb1cddd5142b625f6b7
  Stored in directory: /private/var/folders/2z/f2mhls0j5fndkvswyd3jr1c00000gn/T/pip-ephem-wheel-cache-u1n08jnv/wheels/83/9a/de/0bdafd3d4652d7633c4beb6571bea354de9b735a329ef7c22e
Successfully built pl-core-news-sm
  Attempting uninstall: pl-core-news-sm
    Found existing installation: pl-core-news-sm 2.3.0
    Uninstalling pl-core-news-sm-2.3.0:
      Successfully uninstalled pl-core-news-sm-2.3.0
You should consider upgrading via the '/Users/ppez/anaconda3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/ppez/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

['Aleksander', 'Budek', 'po', '8', 'rok', 'rozstawać', 'się', 'z', 'radiowy', 'trójka']


In [13]:
# Najprostszy odwrócony indeks z lematyzacją


def get_inv_index(documents):
    index = {}
    matching_docs = []
    for d in documents:
        doc = nlp_pl(d['title'])
        for w in doc:
            if w.lemma_ in index:
                index[w.lemma_].append(d)
            else:
                index[w.lemma_]=[d]
        
    return index


inverted_index = get_inv_index(headlines_pl[:10])

for lemma in inverted_index:
    if len(inverted_index[lemma]) > 1:
        print("{} ({})\t{}".format(lemma, len(inverted_index[lemma]), [d['title'] for d in inverted_index[lemma]]))



z (2)	['Aleksandra Budka po 8 latach rozstaje się z radiową Trójką', 'Gabriela Łazarczyk odchodzi z "Gazety Wyborczej"']
. (2)	['Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'Xiaomi przedstawia smartfony Mi10T. Ekrany 6,67 cala i wydajne procesory w cenie od 249 do 649 euro']
i (3)	['Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'Xiaomi przedstawia smartfony Mi10T. Ekrany 6,67 cala i wydajne procesory w cenie od 249 do 649 euro', 'Trzy nowe kanały w Play Now i Play Now TV, w październiku otwarte okno']
medium (2)	['Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'Redaktorzy mediów niezależnych na Białorusi: trwa czystka przestrzeni medialnej']
nowy (2)	['Nowy serial „Król” od 6 listopada w Canal+ (oficjalny zwiastun)', 'Trzy nowe kanały w Play Now i Play Now TV, w październiku otwarte okno']
od (2)	['Nowy serial „Król” od 6 listopada

In [16]:
# Wyszukiwanie w odwróconym indeksie

# Indeksowanie
inverted_index = get_inv_index(headlines_pl[:1000])
print("Distinct lemmas: {}.".format(len(inverted_index)))

# Normalizacja zapytania
user_query = 'media'
doc = nlp_pl(user_query)
print([w.lemma_ for w in doc])


# 'Wyszukiwanie' 
for lemma in [w.lemma_ for w in doc]:
    print(inverted_index[lemma])




Distinct lemmas: 3594.
['medium']
[{'id': 'a289227a9701294ee5cee628e7dd267e', 'language': 'pl', 'source': 'wirtualnemedia.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'url': 'https://www.wirtualnemedia.pl/artykul/senat-odrzucil-sprawozdanie-krrit-za-2019-r-i-informacje-o-dzialalnosci-rady-mediow-narodowych', 'category': ['news']}, {'id': 'cff0c0a0641efbe21a0a55e9efc5c96d', 'language': 'pl', 'source': 'wirtualnemedia.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Redaktorzy mediów niezależnych na Białorusi: trwa czystka przestrzeni medialnej', 'url': 'https://www.wirtualnemedia.pl/artykul/redaktorzy-mediow-niezaleznych-na-bialorusi-trwa-czystka-przestrzeni-medialnej', 'category': ['news']}, {'id': 'abe71863651df31a9cc76f61834781a3', 'language': 'pl', 'source': 'newsweek.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Jarosław, śmielej! Prawicowe media wzywają obóz władzy do 

In [7]:
# Odwrócony indeks nazw własnych?
doc = nlp_pl('Magazyn śledczy Anity Gargas')
print([w.lemma_ for w in doc.ents])

['Anita gargas']


In [8]:
# Instalacja Solr
# Sprawdźmy wersję JAVA VM 

!java -version

java version "1.8.0_191"
Java(TM) SE Runtime Environment (build 1.8.0_191-b12)
Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)


In [9]:
# Pobieramy katalog z konfiguracją indeksów news_pl i wiki_pl

!rm solr-preconf.tar.gz
!gdown https://drive.google.com/uc?id=1CUFilOS3raMmLkzEQeXW2PHxxZF2iusF
!tar -zxvf solr-preconf.tar.gz



rm: solr-preconf.tar.gz: No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1CUFilOS3raMmLkzEQeXW2PHxxZF2iusF
To: /private/tmp/solr-preconf.tar.gz
216MB [00:14, 15.2MB/s] 
x solr-preconf/
x solr-preconf/LUCENE_CHANGES.txt
x solr-preconf/.DS_Store
x solr-preconf/bin/
x solr-preconf/licenses/
x solr-preconf/dist/
x solr-preconf/example/
x solr-preconf/server/
x solr-preconf/docs/
x solr-preconf/contrib/
x solr-preconf/CHANGES.txt
x solr-preconf/NOTICE.txt
x solr-preconf/README.txt
x solr-preconf/LICENSE.txt
x solr-preconf/contrib/.DS_Store
x solr-preconf/contrib/dataimporthandler/
x solr-preconf/contrib/clustering/
x solr-preconf/contrib/jaegertracer-configurator/
x solr-preconf/contrib/langid/
x solr-preconf/contrib/prometheus-exporter/
x solr-preconf/contrib/velocity/
x solr-preconf/contrib/ltr/
x solr-preconf/contrib/dataimporthandler-extras/
x solr-preconf/contrib/analysis-extras/
x solr-preconf/contrib/extraction/
x solr-preconf/contrib/extraction/lib/
x s

x solr-preconf/server/solr-webapp/webapp/partials/threads.html
x solr-preconf/server/solr-webapp/webapp/partials/segments.html
x solr-preconf/server/solr-webapp/webapp/partials/stream.html
x solr-preconf/server/solr-webapp/webapp/partials/logging-levels.html
x solr-preconf/server/solr-webapp/webapp/partials/dataimport.html
x solr-preconf/server/solr-webapp/webapp/partials/cloud.html
x solr-preconf/server/solr-webapp/webapp/partials/query.html
x solr-preconf/server/solr-webapp/webapp/partials/core_overview.html
x solr-preconf/server/solr-webapp/webapp/partials/collections.html
x solr-preconf/server/solr-webapp/webapp/partials/replication.html
x solr-preconf/server/solr-webapp/webapp/partials/alias_overview.html
x solr-preconf/server/solr-webapp/webapp/partials/logging.html
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/
x solr-preconf/server/solr-webapp/webapp/WEB-INF/web.xml
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/hppc-0.8.1.jar
x solr-preconf/server/solr-webapp/weba

x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/hadoop-common-3.2.0.jar
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/lucene-expressions-8.7.0.jar
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/httpmime-4.5.12.jar
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/woodstox-core-asl-4.4.1.jar
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/netty-handler-4.1.50.Final.jar
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/netty-transport-native-unix-common-4.1.50.Final.jar
x solr-preconf/server/solr-webapp/webapp/WEB-INF/lib/netty-transport-native-epoll-4.1.50.Final.jar
x solr-preconf/server/solr-webapp/webapp/img/favicon.ico
x solr-preconf/server/solr-webapp/webapp/img/chosen-sprite@2x.png
x solr-preconf/server/solr-webapp/webapp/img/jstree/
x solr-preconf/server/solr-webapp/webapp/img/loader-light.gif
x solr-preconf/server/solr-webapp/webapp/img/chosen-sprite.png
x solr-preconf/server/solr-webapp/webapp/img/solr.svg
x solr-preconf/server/solr-webapp/webapp/img

x solr-preconf/server/solr-webapp/webapp/img/filetypes/divx.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/pl.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/conf.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/eml.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/file.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/tar.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/sql.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/enc.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/java.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/py.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/rb.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/js.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/xml.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/rar.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/deb.png
x solr-preconf/server/solr-webapp/webapp/img/filetypes/

x solr-preconf/server/lib/ext/log4j-api-2.13.2.jar
x solr-preconf/server/lib/ext/log4j-web-2.13.2.jar
x solr-preconf/server/lib/ext/slf4j-api-1.7.24.jar
x solr-preconf/server/lib/ext/log4j-core-2.13.2.jar
x solr-preconf/server/lib/ext/log4j-slf4j-impl-2.13.2.jar
x solr-preconf/server/lib/ext/jcl-over-slf4j-1.7.24.jar
x solr-preconf/server/lib/ext/log4j-1.2-api-2.13.2.jar
x solr-preconf/server/lib/ext/jul-to-slf4j-1.7.24.jar
x solr-preconf/server/lib/ext/disruptor-3.4.2.jar
x solr-preconf/server/scripts/cloud-scripts/
x solr-preconf/server/scripts/cloud-scripts/zkcli.sh
x solr-preconf/server/scripts/cloud-scripts/zkcli.bat
x solr-preconf/server/scripts/cloud-scripts/snapshotscli.sh
x solr-preconf/server/logs/solr.log
x solr-preconf/server/logs/solr.log.4
x solr-preconf/server/logs/solr.log.3
x solr-preconf/server/logs/solr-8983-console.log
x solr-preconf/server/logs/solr.log.2
x solr-preconf/server/logs/solr.log.5
x solr-preconf/server/logs/solr.log.10
x solr-preconf/server/logs/solr.lo

...

x solr-preconf/example/example-DIH/solr/solr/conf/xslt/example_rss.xsl
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_gl.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_es.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_fi.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_da.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_hu.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_id.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/hyphenations_ga.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/contractions_it.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_ro.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_eu.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_pt.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stopwords_de.txt
x solr-preconf/example/example-DIH/solr/solr/conf/lang/stop

x solr-preconf/example/files/conf/velocity/facet_text_shingles.vm
x solr-preconf/example/files/conf/velocity/facets.vm
x solr-preconf/example/files/conf/velocity/hit.vm
x solr-preconf/example/files/conf/velocity/img/
x solr-preconf/example/files/conf/velocity/browse.vm
x solr-preconf/example/files/conf/velocity/results.vm
x solr-preconf/example/files/conf/velocity/layout.vm
x solr-preconf/example/files/conf/velocity/dropit.js
x solr-preconf/example/files/conf/velocity/jquery.tx3-tag-cloud.js
x solr-preconf/example/files/conf/velocity/img/english_640.png
x solr-preconf/example/files/conf/velocity/img/globe_256.png
x solr-preconf/example/files/conf/velocity/img/france_640.png
x solr-preconf/example/files/conf/velocity/img/germany_640.png
x solr-preconf/example/files/conf/velocity/js/jquery.autocomplete.js
x solr-preconf/example/files/conf/velocity/js/dropit.js
x solr-preconf/example/files/conf/velocity/js/jquery.tx3-tag-cloud.js
x solr-preconf/example/files/browse-resources/velocity/
x s

x solr-preconf/licenses/commons-text-1.6.jar.sha1
x solr-preconf/licenses/jackcess-encrypt-NOTICE.txt
x solr-preconf/licenses/aspectjrt-LICENSE-EPL.txt
x solr-preconf/licenses/curator-framework-2.13.0.jar.sha1
x solr-preconf/licenses/commons-lang-2.6.jar.sha1
x solr-preconf/licenses/asm-8.0.1.jar.sha1
x solr-preconf/licenses/hadoop-minicluster-LICENSE-ASL.txt
x solr-preconf/licenses/fontbox-NOTICE.txt
x solr-preconf/licenses/jackson-dataformat-smile-NOTICE.txt
x solr-preconf/licenses/attributes-binder-1.3.3.jar.sha1
x solr-preconf/licenses/poi-ooxml-schemas-LICENSE-ASL.txt
x solr-preconf/licenses/kerb-client-1.0.1.jar.sha1
x solr-preconf/licenses/bcpkix-jdk15on-LICENSE-BSD_LIKE.txt
x solr-preconf/licenses/zookeeper-jute-NOTICE.txt
x solr-preconf/licenses/jempbox-1.8.16.jar.sha1
x solr-preconf/licenses/snappy-java-LICENSE-ASL.txt
x solr-preconf/licenses/tika-parsers-1.24.1.jar.sha1
x solr-preconf/licenses/calcite-linq4j-1.18.0.jar.sha1
x solr-preconf/licenses/jackcess-3.0.1.jar.sha1
x s

x solr-preconf/licenses/netty-transport-native-unix-common-NOTICE.txt
x solr-preconf/licenses/langdetect-LICENSE-ASL.txt
x solr-preconf/licenses/httpclient-LICENSE-ASL.txt
x solr-preconf/licenses/asm-NOTICE.txt
x solr-preconf/licenses/xmlbeans-LICENSE-ASL.txt
x solr-preconf/licenses/carrot2-mini-NOTICE.txt
x solr-preconf/licenses/http2-server-9.4.27.v20200227.jar.sha1
x solr-preconf/licenses/morfologik-polish-NOTICE.txt
x solr-preconf/licenses/velocity-tools-view-jsp-LICENSE-ASL.txt
x solr-preconf/licenses/randomizedtesting-runner-2.7.2.jar.sha1
x solr-preconf/licenses/parso-LICENSE-ASL.txt
x solr-preconf/licenses/tagsoup-NOTICE.txt
x solr-preconf/licenses/curator-recipes-NOTICE.txt
x solr-preconf/licenses/bcprov-LICENSE-BSD_LIKE.txt
x solr-preconf/licenses/velocity-tools-view-jsp-3.0.jar.sha1
x solr-preconf/licenses/java-libpst-NOTICE.txt
x solr-preconf/licenses/jetty-deploy-9.4.27.v20200227.jar.sha1
x solr-preconf/licenses/metadata-extractor-2.13.0.jar.sha1
x solr-preconf/licenses/co

x solr-preconf/licenses/jetty-NOTICE.txt
x solr-preconf/licenses/jul-to-slf4j-LICENSE-MIT.txt
x solr-preconf/licenses/t-digest-3.1.jar.sha1
x solr-preconf/licenses/poi-LICENSE-ASL.txt
x solr-preconf/licenses/httpmime-LICENSE-ASL.txt
x solr-preconf/licenses/xercesImpl-LICENSE-ASL.txt
x solr-preconf/licenses/opentracing-noop-NOTICE.txt
x solr-preconf/licenses/hadoop-annotations-LICENSE-ASL.txt
x solr-preconf/licenses/jaeger-core-NOTICE.txt
x solr-preconf/licenses/commons-collections-NOTICE.txt
x solr-preconf/licenses/objenesis-NOTICE.txt
x solr-preconf/licenses/rrd4j-NOTICE.txt
x solr-preconf/licenses/tika-parsers-LICENSE-ASL.txt
x solr-preconf/licenses/metrics-jvm-4.1.5.jar.sha1
x solr-preconf/licenses/commons-csv-NOTICE.txt
x solr-preconf/licenses/netty-buffer-NOTICE.txt
x solr-preconf/licenses/metrics-core-LICENSE-ASL.txt
x solr-preconf/licenses/metrics-jetty-LICENSE-ASL.txt
x solr-preconf/licenses/mockito-core-2.23.4.jar.sha1
x solr-preconf/licenses/antlr4-runtime-LICENSE-BSD.txt
x s

In [10]:
# Uruchamianie instancji: 

!./solr-preconf/solr-preconf/bin/solr stop -all
!./solr-preconf/bin/solr  start -force

/bin/sh: ./solr-preconf/solr-preconf/bin/solr: No such file or directory
*** [WARN] *** Your open file limit is currently 256.  
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
*** [WARN] ***  Your Max Processes Limit is currently 2784. 
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
Waiting up to 180 seconds to see Solr running on port 8983 [|] [/] [-] [\] [|] [/] [-]
Started Solr server on port 8983 (pid=21409). Happy searching!

 [\]  

In [ ]:
# Dla zainteresowanych: instalacja z wersji dystrybucyjnej Solra
# Pobieramy jedną ostatnich wersji Solra

!wget https://apache.mirrors.tworzy.net/lucene/solr/8.7.0/solr-8.7.0.tgz 
    
# Rozpakowujemy archiwum

!tar -xzf solr-8.7.0.tgz
!rm solr-8.7.0.tgz

# Kopiujemy dostarczoną konfigurację indeksu do odpowiedniego podkatalogu:

!cp -R ./solr-preconf/server/solr/news_pl ./solr-8.7.0/server/solr/

# Aby utworzyć indeks dla danych z Wikipedii kopiujemy do odpowiednio nazwanego folderu:

!cp -R ./solr-preconf/server/solr/news_pl ./solr-8.7.0/server/solr/wiki_pl

# Uruchamianie instancji: 
!solr-8.7.0/bin/solr start

# Uruchamianie instancji w środowisku Google Colab
#!/content/solr-8.7.0/bin/solr start -force

# Następnie należy zarejestrować indeks news_pl, np. z konsoli Solr 
# (http://localhost:8983/solr/#/ -> Core Admin -> Add Core -> name:news_pl instanceDir:news_pl)

# Oba indeksy powinny być widoczne w konsoli: http://localhost:8983/solr/#/ 

--2020-11-19 20:15:48--  https://apache.mirrors.tworzy.net/lucene/solr/8.7.0/solr-8.7.0.tgz
Resolving apache.mirrors.tworzy.net (apache.mirrors.tworzy.net)... 51.83.241.43
Connecting to apache.mirrors.tworzy.net (apache.mirrors.tworzy.net)|51.83.241.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200805960 (192M) [application/x-gzip]
Saving to: ‘solr-8.7.0.tgz’

solr-8.7.0.tgz      100%[===================>] 191.50M  7.65MB/s    in 38s     

2020-11-19 20:16:26 (5.07 MB/s) - ‘solr-8.7.0.tgz’ saved [200805960/200805960]



In [12]:
# Metody do indeksowania dokumentów przez HTTP w uprzednio zdefiniowanym schemacie indeksu Solr

import json
import requests

def index_documents(solr_url, docs):
    bytes = json.dumps(docs).encode(encoding='UTF-8')
    if(len(docs)> 0):
        bytes = json.dumps(docs).encode(encoding='UTF-8')
        try:
            res = requests.post(url=solr_url+'/update/json/docs',
                                data=bytes,
                                headers={'Content-Type': 'application/octet-stream'})
            solr_url+'update?commit=true'
            print("Update response: {}".format(res))
        except Exception as e:
            print(str(e))

def index_by_batch(solr_url, docs, batch_size):
    print("USING CORE: {}" .format(solr_url))
    doc_buffer=[]
    for doci, doc in enumerate(docs):
        doc_buffer.append(doc)
        if len(doc_buffer)%batch_size==0:
            index_documents(solr_url = solr_url, docs=doc_buffer)
            print('Adding doc {} to index'.format(doci+1))
            doc_buffer.clear()
    print('Adding doc {} to index'.format(doci+1))
    index_documents(solr_url = solr_url, docs=doc_buffer)
    print('Done indexing.'.format(doci+1))
    
    requests.get(solr_url +'/update?commit=true')
 

        

In [13]:
def delete_all_documents(solr_url):
    """
    Wywołanie równoważne z:
    !curl "http://localhost:8983/solr/news_pl/update?commit=true" -H "Content-Type: text/xml" --data-binary '<delete><query>*:*</query></delete>'
    """
    try:
        print(solr_url+'/update?commit=true')
        res = requests.get(url=solr_url+'/update?commit=true',
                                data='<delete><query>*:*</query></delete>'.encode(encoding='UTF-8'),
                                headers={'Content-Type': 'text/xml'})
        print(res)
    except Exception as e:
        print(str(e))

In [14]:
# Przykłady indeksowania dokumentów    

doc1 = {'id':'136f7046b8c9bf01ba3d3fd331180b79', 
        'language':'pl', 
        'source':'dziennik.pl',
        'published':'2011-01-23T18:31:00Z',
        'category':['news'],
        'title':'Tajemnicza historia chińskiego niewidzialnego samolotu', 
        'url':'https://wiadomosci.dziennik.pl/swiat/artykuly/318888,tajemnicza-historia-chinskiego-niewidzialnego-samolotu.html'
       }

# Opcjonalnie możemy najpierw usunąć wszystkie dokumenty
print("Clearing the index...")

s_url = 'http://localhost:8983/solr/news_pl'
delete_all_documents(solr_url = s_url)

# Dodajemy testowo 1 dokument:
index_by_batch(solr_url = s_url, docs=[doc1], batch_size=10)


Clearing the index...
http://localhost:8983/solr/news_pl/update?commit=true
<Response [200]>
USING CORE: http://localhost:8983/solr/news_pl
Adding doc 1 to index
Update response: <Response [200]>
Done indexing.


In [15]:
# Dodajemy ponad 100 tys. dokumentów

delete_all_documents(solr_url = s_url)
headlines_pl = read_json_documents('news_pl.jsonl')
print("Headlines to index {}.".format(len(headlines_pl)))

index_by_batch(solr_url=s_url, docs = headlines_pl, batch_size=10000)


http://localhost:8983/solr/news_pl/update?commit=true
<Response [200]>
Headlines to index 116056.
USING CORE: http://localhost:8983/solr/news_pl
Update response: <Response [200]>
Adding doc 10000 to index
Update response: <Response [200]>
Adding doc 20000 to index
Update response: <Response [200]>
Adding doc 30000 to index
Update response: <Response [200]>
Adding doc 40000 to index
Update response: <Response [200]>
Adding doc 50000 to index
Update response: <Response [200]>
Adding doc 60000 to index
Update response: <Response [200]>
Adding doc 70000 to index
Update response: <Response [200]>
Adding doc 80000 to index
Update response: <Response [200]>
Adding doc 90000 to index
Update response: <Response [200]>
Adding doc 100000 to index
Update response: <Response [200]>
Adding doc 110000 to index
Adding doc 116056 to index
Update response: <Response [200]>
Done indexing.


In [16]:
#Przykłady odpytywania indeksu z poziomu Pythona 


from urllib.request import urlopen

import json
s_url = 'http://localhost:8983/solr/news_pl'

def print_response(response, fields=['id','title','score']):
    print("\n{}\nTotal documents found: {}\n{}\n"
          .format('-'*25,response['response']['numFound'],'-'*25))
    # Print the name of each document.
    for di, document in enumerate(response['response']['docs']):
        print("{}. {}\t{}\t{}".format(di+1, *([document[f] for f in fields])))
        
    if 'facet_counts' in response:
        print("\n FACETS: \n")
        print(response['facet_counts']['facet_fields'])


In [17]:
# 1. Proste zapytanie o 10 dokumentów zawierających dowolną formę słowa 'samolot'
connection = urlopen(s_url+'/select?q=title:samolot&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


-------------------------
Total documents found: 128
-------------------------

1. wnp.pl	Czas na mniejsze samoloty?	4.300211
2. interia.pl	Stworzył własny samolot elektryczny	4.300211
3. wnp.pl	Nieszczepieni samolotem nie polecą	4.300211
4. tvp.info	Kraksa samolotów. Pięć osób zginęło	4.1164083
5. wnp.pl	Lufthansa uziemi jeszcze więcej samolotów	4.1164083
6. interia.pl	Niezidentyfikowany samolot zauważony nad Kalifornią	4.1164083
7. onet.pl	Awaryjne lądowanie samolotu AN-124	4.1164083
8. onet.pl	Awaryjne lądowanie samolotu AN-124	4.1164083
9. polsatnews.pl	Zderzenie samolotów we Francji. Są ofiary	3.9476738
10. tvp.info	Niemieccy mechanicy odmówili serwisu samolotu Łukaszenki	3.9476738


In [20]:
# Składnia DisMax
# https://lucene.apache.org/solr/guide/8_7/the-dismax-query-parser.html

import urllib

# 1.Zapytanie o 10 dokumentów zawierających dowolną formę słowa 'samolot' lub 'lotniczy'
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('title:samolot OR title:lotniczy')
    +'&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


-------------------------
Total documents found: 235
-------------------------

1. gazeta.pl	Samoloty wracają nad Wisłę. Lotniczy pokaz dla upamiętnienia 102. rocznicy odzyskania niepodległości	6.3987446
2. polsatnews.pl	Najdłuższe połączenie lotnicze świata wraca do rozkładu lotów. Samoloty pokonają ponad 15 tys. km	6.003709
3. tvn24.pl	Do samolotu wejdą tylko zaszczepieni. To pierwsza linia lotnicza, która zdecydowała się na taki krok	5.823934
4. wp.pl	Szczepionka na COVID warunkiem wejścia na pokład samolotu? Pierwsza linia lotnicza zdecydowała się na taki krok	5.654613
5. dzienniklodzki.pl	Koronawirus. Szczepienie może być wymagane do lotu samolotem. Linia Qantas to pierwsza linia lotnicza, która zdecydowała się na taki krok	5.0655265
6. wnp.pl	Czas na mniejsze samoloty?	4.300211
7. interia.pl	Stworzył własny samolot elektryczny	4.300211
8. wnp.pl	Nieszczepieni samolotem nie polecą	4.300211
9. tvp.info	Kraksa samolotów. Pięć osób zginęło	4.1164083
10. wnp.pl	Lufthansa uziemi jeszc

In [21]:
# Wzbogacanie indeksu o jednostki nazewnicze (Zob. Moduł 6.4)

import spacy
nlp_pl = spacy.load("pl_core_news_sm")

def annotate_documents(documents):
    for di, d in enumerate(documents):
        doc = nlp_pl(d["title"])
        named_ents=["{}___{}".format(w.lemma_, w.label_) for w in doc.ents]
        if len(named_ents) > 0:
           d["ents"]=named_ents
           d["ent_types"]=[w.label_ for w in doc.ents]
        if((di)%1000 == 0):
            print("Annotated {} of {} documents.".format(di+1,len(documents)))
    return documents

headlines_pl = read_json_documents('news_pl.jsonl')
print("Headlines to index {}.".format(len(headlines_pl)))

annotated_headlines = annotate_documents(headlines_pl[0:10])

for ah in annotated_headlines:
    print("{}\t{}\n".format(ah['ents'], ah['title']))

#s_url = 'http://localhost:8983/solr/news_pl'
#delete_all_documents(solr_url = s_url)

# Dodajemy testowo 1 dokument:
#index_by_batch(solr_url = s_url, docs=[doc1], batch_size=10)

Headlines to index 116056.
Annotated 1 of 10 documents.
['Aleksander Budek___persName', 'trójka___orgName']	Aleksandra Budka po 8 latach rozstaje się z radiową Trójką

['senat___orgName', 'krrit___orgName', '2019 r .___date', 'rada medium narodowy___orgName']	Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych

['6 listopad___date', 'canal+___placeName']	Nowy serial „Król” od 6 listopada w Canal+ (oficjalny zwiastun)

['plusa___orgName']	Technologia 5G dostępna także dla abonentów Plusa na Kartę

['Białoruś___placeName']	Redaktorzy mediów niezależnych na Białorusi: trwa czystka przestrzeni medialnej

['Gabriel łazarczyk___persName', '" gazeta wyborczy "___orgName']	Gabriela Łazarczyk odchodzi z "Gazety Wyborczej"

['tomasz jankowski___persName', 'zarząd zasobu komunalnego w Wrocław___orgName']	Tomasz Jankowski rzecznikiem prasowym Zarządu Zasobu Komunalnego we Wrocławiu



KeyError: 'ents'

In [ ]:
# Reindeksujemy dokumenty

annotated_headlines = annotate_documents(headlines_pl[0:500000])

s_url = 'http://localhost:8983/solr/news_pl'

delete_all_documents(solr_url = s_url)
print("Headlines to index {}.".format(len(annotated_headlines)))

index_by_batch(solr_url=s_url, docs = annotated_headlines, batch_size=10000)


Annotated 1 of 116056 documents.
Annotated 1001 of 116056 documents.
Annotated 2001 of 116056 documents.
Annotated 3001 of 116056 documents.
Annotated 4001 of 116056 documents.
Annotated 5001 of 116056 documents.
Annotated 6001 of 116056 documents.
Annotated 7001 of 116056 documents.
Annotated 8001 of 116056 documents.
Annotated 9001 of 116056 documents.
Annotated 10001 of 116056 documents.
Annotated 11001 of 116056 documents.
Annotated 12001 of 116056 documents.
Annotated 13001 of 116056 documents.
Annotated 14001 of 116056 documents.
Annotated 15001 of 116056 documents.
Annotated 16001 of 116056 documents.
Annotated 17001 of 116056 documents.
Annotated 18001 of 116056 documents.
Annotated 19001 of 116056 documents.
Annotated 20001 of 116056 documents.
Annotated 21001 of 116056 documents.
Annotated 22001 of 116056 documents.
Annotated 23001 of 116056 documents.
Annotated 24001 of 116056 documents.
Annotated 25001 of 116056 documents.
Annotated 26001 of 116056 documents.
Annotated 2700

In [ ]:
# Zapytanie o tytuły zawierające nazwę miejsca

import urllib

# 1.Zapytanie o 10 dokumentów zawierających dowolną formę słowa 'samolot' a zarazem nazwę miejsca
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('title:samolot AND ent_types:placeName')
    +'&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


---------------------------
Total documents found: 5
---------------------------

1. onet.pl	Wrocław: 36 hiszpańskich studentów z koronawirusem. Przylecieli jednym samolotem	3.611897
2. rmf24.pl	Grecja wprowadza obostrzenia. Biura podróży wysyłają puste samoloty po Polaków	3.4973776
3. gazeta.pl	Robert Lewandowski będzie błyszczał w Szwajcarii. Z kolejnym trofeum w samolocie	3.3907225
4. wp.pl	18-letnia Julia poleciała do USA rządowym samolotem. "Jesteśmy z Tobą!"	3.3907225
5. wp.pl	Koronawirus. 41-letnia Polka zatrzymana na lotnisku Schiphol. Gryzła i kopała załogę samolotu	3.1979733


In [ ]:
# Zapytanie o tytuły zawierające nazwę miejsca

import urllib

# 1.Zapytanie o 10 dokumentów zawierających wzmianki osób zaczynające się od ciągu 'Gabriel'
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('ents:Gabriel*___persName')
    +'&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


---------------------------
Total documents found: 12
---------------------------

1. wirtualnemedia.pl	Gabriela Łazarczyk odchodzi z "Gazety Wyborczej"	1.0
2. interia.pl	Gabriela Morawska-Stanecka o nowym ministrze edukacji: Skandaliczna nominacja	1.0
3. press.pl	Gabriela Dworniak przestała kierować Agencją Kreacji Rozrywki i Oprawy TVP	1.0
4. rp.pl	Gabriela Morawska-Stanecka: Wyrok TK wynika z tchórzostwa	1.0
5. interia.pl	Rafał i Gabriel z "Królowych życia" zostali okradzeni	1.0
6. plotek.pl	"Królowe życia". Rafał i Gabriel zostali okradzeni. "Nie mieliśmy jak wejść do domu"	1.0
7. interia.pl	Rafał i Gabriel z "Królowych życia" zostali okradzeni! Przeżyli dramat!	1.0
8. plotek.pl	"Królowe życia". Dagmara Kaźmierska kpi ze związku Gabriela i Rafała: "To jest gorsze od tego, czym ja się zajmowałam"	1.0
9. kobieta.wp.pl	Ewa Bem w rozsypce. Jej córka Gabriela podjęła bardzo trudną decyzję	1.0
10. gazeta.pl	Nie żyje Gabriel Chmura, dyrektor artystyczny Teatru Wielkiego. "Muzyka już nigd

In [ ]:
# Agregacja wszystkich pasujących dokumentów po typach nazw własnych

import urllib

# 1.Zapytanie o 5 dokumentów zawierających lemat 'Brazylia'
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('title:brazylia')
    +'&wt=json&rows=5&fl=*,score'
    +'&facet.field=ent_types&facet=on')
response = json.load(connection)
print_response(response=response, fields=['source','title','ents'])




---------------------------
Total documents found: 112
---------------------------

1. tvn24.pl	Ambasador odwołana z Brazylii	['Brazylia___placeName']
2. interia.pl	Brazylia: Znaczny spadek przyrostu zakażeń	['brazylia___persName']
3. rp.pl	Minister zdrowia Brazylii zakażony koronawirusem	['Brazylia___placeName']
4. tvn24.pl	Nie ma mocnych na Brazylię	['Brazylia___persName']
5. wnp.pl	Brazylia: 27 750 nowych przypadków koronawirusa	['brazylia___persName']

 FACETS: 

{'ent_types': ['persName', 87, 'placeName', 44, 'geogName', 14, 'orgName', 7, 'date', 3, 'time', 0]}
